In [3]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
 
from bertopic import BERTopic
from umap import UMAP
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import re
from matplotlib.animation import FuncAnimation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Export-JRC-Data-Catalogue.xlsx')
df2= df.copy()

In [5]:
df

,ID,Name,Website,Description,Intéret économique et public du projet (A>B>C),code annotateur,Geographical extent,Geographic coverage\nCountry,Geographic coverage \nNUTS 2021,Responsible\nOrganization,...,Improved management of public resources,Increased quality of processes and systems,Better collaboration and better communication,Reduced or eliminate the risk of corruption and abuse of the law by public servants,"Enabled greater fairness, honesty, equality",Open government capabilities,Increased transparency of public sector operations,Increased public participation in government actions and policy making,Improved public control and influence on government actions and policies,Source
0,1,AMS - public employment service,Algorithmic Profiling of Job Seekers in Austri...,"As of 2020, the Public Employment Service Aust...",NaN,1.0,National,Austria,AT,Public Employment Service Austria,...,NaN,NaN,NaN,NaN,NaN,Open government capabilities,NaN,NaN,x,(https://www.derstandard.at/story/200011948693...
1,2,Mona - Public chatbot for companies on the sub...,https://www.usp.gv.at/Portal.Node/usp/public,"As of 2020, the Public Employment Service Aust...",NaN,2.0,National,Austria,AT,"Unternehmensservice Portal, Austria",...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,https://oecd-opsi.org/covid-response/infopoint...
2,3,CitizenLab - Youth for Climate,https://youth4climate.be/en/,"As of 2020, the Public Employment Service Aust...",NaN,3.0,National,Belgium,BE,"CitizenLab, Belgium",...,NaN,NaN,NaN,NaN,NaN,Open government capabilities,x,x,x,https://www.citizenlab.co/blog/product-update/...
3,4,Walloon - Agricultural subsidy monitoring with...,https://inspire.ec.europa.eu/sites/default/fil...,Walloon uses geo AI with satellite imagery to ...,NaN,4.0,Regional,Belgium,BE2,Département de l'Agriculture,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,ELISE INSPIRE webinar 3-6-2020 https://inspire...
4,5,AcPaas - Technical procurement documents compa...,https://acpaas.digipolis.be/nl/home,The city of Antwerp has a platform called Antw...,NaN,5.0,Local,Belgium,BE21,Digipolis & city of Antwerp,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,https://acpaas.digipolis.be/nl/product/NLP-eng...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,682,Chatbot Comune di Collegno (TO),https://www.comune.collegno.to.it/home,Chatbot to support citizen in assistance on Ci...,NaN,NaN,Local,Italy,ITC11,Collegno Municipality,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,Comune di Collegno
682,683,MARVEL project in Trento: monitoring crowd in ...,https://www.marvel-project.eu/smart-region-the...,Project Pilot in Trento is focusing: \nI. Crow...,NaN,NaN,Across Countries,Italy,ITH20,Foundation for Research and Technology Hellas,...,x,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,MARVEL (Trento)
683,684,Chatbot Covibot Provincia Trento (TN),Covibot Trento,"Chatbot that provides information on tampons, ...",NaN,NaN,Regional,Italy,ITH20,Autonomous Province of Trento - Health Agency,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,Covibot Trento
684,685,Belgian city uses artificial intelligence to t...,Belgian city uses artificial intelligence to t...,The Belgian city of Genk is working with Nokia...,NaN,NaN,Local,Belgium,BE22,City of Genk,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,Belgian city uses artificial intelligence to t...


In [6]:
df = df[['ID', 'Description']]

In [7]:

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

preprocessed_descriptions = []

for description in df['Description']:
    # Tokenisation des mots
    tokens = word_tokenize(description)

    # Suppression des stopwords et lemmatisation
    filtered_tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]

    # Rejoindre les tokens filtrés en une phrase prétraitée
    preprocessed_description = ' '.join(filtered_tokens)
    preprocessed_descriptions.append(preprocessed_description)

# Ajouter la colonne prétraitée au DataFrame
df['Description'] = preprocessed_descriptions

In [8]:

umap_model = UMAP(random_state=42,n_components=2)
model = BERTopic(language="english",umap_model=umap_model)
topics, probabilities = model.fit_transform(df['Description'])

In [9]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,302,-1_data_service_project_system,"[data, service, project, system, information, ...",[support system classification incidences/comp...
1,0,59,0_patient_health_treatment_hospital,"[patient, health, treatment, hospital, clinica...",[faith aim provide artificial intelligence app...
2,1,55,1_road_traffic_vehicle_camera,"[road, traffic, vehicle, camera, city, transpo...","[flemish motorway , road lighting needed . spe..."
3,2,47,2_chatbot_service_answer_question,"[chatbot, service, answer, question, available...","[chatbots use customer service since 2017 , ye..."
4,3,30,3_police_crime_software_burglary,"[police, crime, software, burglary, report, mu...",[filing fake police report quite common spain ...
5,4,29,4_ice_agricultural_image_satellite,"[ice, agricultural, image, satellite, food, da...",[estonian agricultural register information bo...
6,5,25,5_fraud_system_risk_financial,"[fraud, system, risk, financial, tax, detect, ...",[ai system supporting fraud prevention . consi...
7,6,25,6_school_child_social_process,"[school, child, social, process, system, paren...","[2010 , slovenian government introduced e-soci..."
8,7,19,7_water_consumption_flood_forecast,"[water, consumption, flood, forecast, data, le...",[monitoring water treatment plant kartuzy incl...
9,8,17,8_legal_law_court_text,"[legal, law, court, text, data, reasoning, sta...",[calculemus flint method interpreting source a...


In [10]:
# Visualize top topic keywords
model.visualize_barchart(top_n_topics=12)
# Save the plot as an image file (e.g., PNG, JPEG, SVG)



In [11]:
# Visualiser la distance intersujet 

model.visualize_topics()


In [12]:
# Visualize connections between topics using hierachical clustering
model.visualize_hierarchy(top_n_topics=10)

In [13]:
topic_words=model.get_topic_info()
list_name=[]
for i in topic_words['Name']:

    i=i.split('_')
    i=' '.join(i)
    i = re.sub(r'\d+', '', i)
    i = re.sub(r'-', '', i)

    list_name.append(i)

    
topic_words['Name']=list_name
df['Description']=df['Description'].astype('str')+' '+df2['Name'].astype('str')



In [14]:
df['Description']=df['Description'].astype('str')
umap_model2 = UMAP(random_state=42,n_components=2)
model2 = BERTopic(language="english",umap_model=umap_model2)
topics, probabilities = model2.fit_transform(df['Description'])

In [15]:
model2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,175,-1_system_data_of_ai,"[system, data, of, ai, project, using, use, se...","[try control , outsmart . logic danish high sc..."
1,0,116,0_road_traffic_system_city,"[road, traffic, system, city, camera, vehicle,...","[flemish motorway , road lighting needed . spe..."
2,1,99,1_chatbot_service_customer_information,"[chatbot, service, customer, information, virt...",[finnish tax administration published joint ch...
3,2,80,2_data_ai_process_platform,"[data, ai, process, platform, learning, public...","[january 2018 , regional government catalonia ..."
4,3,62,3_patient_health_treatment_hospital,"[patient, health, treatment, hospital, clinica...",[faith aim provide artificial intelligence app...
5,4,32,4_agricultural_ice_satellite_image,"[agricultural, ice, satellite, image, forest, ...","[together development partner , environmental ..."
6,5,22,5_financial_tax_fraud_audit,"[financial, tax, fraud, audit, risk, aid, anal...",[ministry finance ’ financial administration u...
7,6,21,6_water_energy_consumption_system,"[water, energy, consumption, system, network, ...",[sensor analyse water consumption order optimi...
8,7,21,7_police_crime_burglary_software,"[police, crime, burglary, software, policing, ...",[state criminal police office ( lka ) north rh...
9,8,17,8_law_legal_regulation_court,"[law, legal, regulation, court, justice, lawye...","[court cassation , conjunction ministry justic..."


In [16]:
model2.visualize_barchart(top_n_topics=12)

In [17]:
model2.visualize_topics()

In [18]:
ds = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Export-JRC-Data-Catalogue.xlsx')

In [20]:
ds = ds[['Description','Geographic coverage\nCountry']]

In [21]:
occurrences = ds['Geographic coverage\nCountry'].value_counts()
mask = occurrences >= 41
ds = ds[ds['Geographic coverage\nCountry'].isin(occurrences[mask].index)]

In [22]:
ds

,Description,Geographic coverage\nCountry
23,In the Estonian Agricultural Registers and Inf...,Estonia
24,Artificial Intelligence (AI) customer service ...,Estonia
25,"In 2015, the Estonian Health Insurance Fund st...",Estonia
26,The speech recognition system helps transcribe...,Estonia
27,"The virtual assistant called ITI, created in c...",Estonia
...,...,...
678,Image sensors with artificial intelligence pro...,Italy
681,Chatbot to support citizen in assistance on Ci...,Italy
682,Project Pilot in Trento is focusing: \nI. Crow...,Italy
683,"Chatbot that provides information on tampons, ...",Italy


In [25]:
num_of_topics = 1

In [26]:
ds['similar_topics'] = ds['Description'].apply(lambda x: model2.find_topics(x, top_n=num_of_topics)[0])

In [27]:
# Obtenez les topics avec leurs mots clés
topics = model2.get_topics()

# Parcourez les topics et attribuez des noms
topic_names = {}
for topic_id, words in topics.items():
    topic_name = "Topic " + str(topic_id)  # Nom par défaut : Topic 1, Topic 2, etc.
    # Remplacez le nom du topic par un nom significatif si nécessaire
    if topic_id == 0:
        topic_name = "Traffic"
    elif topic_id == 1:
        topic_name = "Chatbot"
    elif topic_id == 2:
        topic_name = "System"
    elif topic_id == 3:
        topic_name = "Sante"
    elif topic_id == 4:
        topic_name = "Nature"
    elif topic_id == 5:
        topic_name = "Finance"
    elif topic_id == 6:
        topic_name = "Energie"
    elif topic_id == 7:
        topic_name = "Securité"
    elif topic_id == 8:
        topic_name = "Loi"
    elif topic_id == 9:
        topic_name = "Education"
    elif topic_id == 10:
        topic_name = "Emploi"
    elif topic_id == 11:
        topic_name = "Traitement de texte"
    elif topic_id == -1:
        topic_name = "IA"
    # Ajoutez le nom du topic au dictionnaire des noms de topic
    topic_names[topic_id] = topic_name

# Appliquez les noms de topic à votre dataset
ds['topic_names'] = ds['similar_topics'].apply(lambda x: [topic_names[topic] for topic in x])

In [28]:
ds

,Description,Geographic coverage\nCountry,similar_topics,topic_names
23,In the Estonian Agricultural Registers and Inf...,Estonia,[4],[Nature]
24,Artificial Intelligence (AI) customer service ...,Estonia,[1],[Chatbot]
25,"In 2015, the Estonian Health Insurance Fund st...",Estonia,[3],[Sante]
26,The speech recognition system helps transcribe...,Estonia,[11],[Traitement de texte]
27,"The virtual assistant called ITI, created in c...",Estonia,[1],[Chatbot]
...,...,...,...,...
678,Image sensors with artificial intelligence pro...,Italy,[0],[Traffic]
681,Chatbot to support citizen in assistance on Ci...,Italy,[1],[Chatbot]
682,Project Pilot in Trento is focusing: \nI. Crow...,Italy,[0],[Traffic]
683,"Chatbot that provides information on tampons, ...",Italy,[1],[Chatbot]


In [29]:
ds =  ds[['Geographic coverage\nCountry','topic_names']]

In [30]:
ds

,Geographic coverage\nCountry,topic_names
23,Estonia,[Nature]
24,Estonia,[Chatbot]
25,Estonia,[Sante]
26,Estonia,[Traitement de texte]
27,Estonia,[Chatbot]
...,...,...
678,Italy,[Traffic]
681,Italy,[Chatbot]
682,Italy,[Traffic]
683,Italy,[Chatbot]


In [31]:
ds['Geographic coverage\nCountry'] = ds['Geographic coverage\nCountry'].apply(lambda x: x[0] if isinstance(x, list) else x)
ds['topic_names'] = ds['topic_names'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [32]:
ds

,Geographic coverage\nCountry,topic_names
23,Estonia,Nature
24,Estonia,Chatbot
25,Estonia,Sante
26,Estonia,Traitement de texte
27,Estonia,Chatbot
...,...,...
678,Italy,Traffic
681,Italy,Chatbot
682,Italy,Traffic
683,Italy,Chatbot


In [37]:
ds = ds.drop(ds[ds['topic_names'] == 'IA'].index)

In [40]:
ds = ds.rename(columns={'Geographic coverage\nCountry': 'Country'})

In [43]:
ds.to_csv('/content/drive/MyDrive/Colab Notebooks/data1.csv', index=False)